In [1]:
import pandas as pd
import numpy as np
import re

# Pre-processamento
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Classificadores
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 

In [2]:
data = pd.read_csv('data/data_estag_ds.tsv', sep='\t')

In [3]:
data.head(10)

,ID,TITLE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...
3,1042568,Car Center - Calesita
4,1042584,Donka Trem com Som - Calesita
5,1043696,Interfone HDL AZ-01 Branco
6,1940486,Smartphone LG Optimus L7 II Dual P716 Preto co...
7,2277815,Smartphone Samsung Galaxy S4 Mini Duos Preto c...
8,2961097,Smartphone Samsung Galaxy S5 SM-G900M Branco ...
9,2961099,Smartphone Samsung Galaxy S5 SM-G900M Dourado...


In [4]:
data.shape

(2606, 2)

In [5]:
# Removendo o atributo ID (irrelevante)
ids = data['ID']
data.drop(['ID'], axis=1, inplace=True)

In [6]:
# Inicializando atributos target com 0.0
data['SMARTPHONE'] = np.zeros((len(data)))

# Parte 1: Anotação dos dados
Objetivando a aplicação de um algoritmo supervisionado, foram classificados manualmente os dados entre 1.0 (Smartphone) e 0.0 (Não-Smartphone) e depois filtrados os possíveis casos indesejáveis. Tal tarefa é relativamente simples, pois os intervalos onde há smartphones são facilmente identificáveis. Caso os dados estivessem muito embaralhados e/ou o conjunto de dados fosse muito grande, outras técnicas poderiam ser usadas (i.e. clustering, regex).

In [7]:
# Manualmente selecionando os intervalos onde existem smartphones
data['SMARTPHONE'].iloc[6:39] = 1.0
data['SMARTPHONE'].iloc[2035:] = 1.0

# Nota: Há alguns smartwatches nesse intervalo.
#       Além disso, pode ser que haja algum smartphone que não esteja nesse intervalo.
#       Esses casos serão tratados a seguir

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
# Casos de Smartwatches classificados como 1.0
for i, example in data[data['SMARTPHONE'] == 1.0]['TITLE'].iteritems():
    if re.findall('watch|rel[ó|o]gio', example.lower()):
        print(i, example)
        data['SMARTPHONE'].iloc[i] = 0.0

2207 Apple Watch Série 3 42mm Cores E Edição  Nike+ Em Estoque
2227 Apple Watch Series 3 42mm Nike + Gps Prova D'água Lacrado
2244 Apple Watch Series 3 42mm Gps Garantia Apple + Brinde
2253 Relógio Samsung Gear S3 Sm-r770 Classic Wifi Bluetooth
2288 Apple Watch Series 3  42mm Nike + Gps Prova D'água Lançameto
2289 Apple Watch Series 3 42mm Nike +  Gps Prova D'água
2298 Apple Watch Series 3 38mm Nike + Gps Prova D'água
2301 Apple Watch Nike+ Séries 3 42mm Lacrado
2302 Apple Watch Nike+ Séries 3 38mm Lacrado Novo
2303 Apple Watch Nike+ Séries 3 38mm Lacrado
2309 Apple Watch Series 3 38mm Nike + Gps Prova D'água Lacrado
2314 Apple Watch Série 3 42mm Gps + 2 Pulseiras Extras
2333 Relógio Samsung S3 Frontier + 2 Pulseiras Extras
2345 Apple Watch Series 3 42mm Lacrado + Brinde
2353 Apple Watch Nike+ Séries 3 42mm Lacrado + Envio Imediato
2380 Apple Watch Series 3 38mm Lacrado Nf + Brinde
2417 Smartwatch Samsung Gear S3 Frontier 1.3 Bluetooth - Original
2597 Relógio Smartwatch Samsung Gear Sm

In [9]:
# Casos de smartphones classificados como 0.0
for i, example in data[data['SMARTPHONE'] == 0.0]['TITLE'].iteritems():
    if re.findall('^smartphone|^celular', example.lower()):
        print(i, example)
        
# Somente 2 casos
data['SMARTPHONE'].iloc[1892:1894] = 1.0

# Nota: Foram encontrados alguns celulares, mas que não são smartphones. Estes foram desconsiderados.

548 Celular Multilaser Up- Branco/Rosa
775 Celular Blu Samba Jr Q53 Prata Dual Chip Silver
1886 Celular Up 2chip Quad Cam Mp3/4 Fm Branco/Rosa
1892 Smartphone Samsung Galaxy J2 Prime TV Dourado com 8GB, Dual chip, Tela 5", TV Digital, 4G, Câmera 8MP, Android 6.0 e Processador Quad Core de 1.4 Ghz
1893 Smartphone Samsung Galaxy J2 Prime TV Preto com 8GB, Dual chip, Tela 5", TV Digital, 4G, Câmera 8MP, Android 6.0 e Processador Quad Core de 1.4 Ghz


In [10]:
data['SMARTPHONE'].value_counts()

0.0    2018
1.0     588
Name: SMARTPHONE, dtype: int64

In [11]:
data['SMARTPHONE'].mean()
# Classes estão desbalanceadas

0.2256331542594014

# Parte 2: Pré-processamento

In [12]:
# Adicionando pontuações às stopwords em portugues
pontuacoes = ['-', '.', ',', '(', ')']
pt_stopwords = stopwords.words('portuguese')

for pontuacao in pontuacoes:
    pt_stopwords.append(pontuacao)

In [13]:
vect = CountVectorizer(stop_words=pt_stopwords)
vect = CountVectorizer().fit(data['TITLE'])

data_vect = vect.transform(data['TITLE'])

# Parte 3: Seleção do Modelo e Validação

In [42]:
SVM = {'name': 'svm',
       'classifier': SVC(),
       'parameters': {'kernel': ['linear', 'rbf'],
       'C': [0.1, 1.0, 5, 10, 15],
       'gamma': [0.01, 0.1, 1.0, 5.0, 10],}}

DTC = {'name': 'decision_tree',
       'classifier': DecisionTreeClassifier(),
       'parameters': {'max_depth': [None, 5, 10, 20, 30],}}

Bernoulli = {'name': 'bernoulli_naive_bayes',
             'classifier': BernoulliNB(),
             'parameters': {'alpha': [1.0, 2.0, 5.0]}}
            
Multinomial = {'name': 'multinomial_naive_bayes',
               'classifier': MultinomialNB(),
               'parameters': {'alpha': [1.0, 2.0, 5.0]}}

classificadores = [SVM, DTC, Bernoulli, Multinomial]

In [35]:
# Encontra os melhores parâmetros para cada classificador

resultados = {}
for classificador in classificadores:
    gs = GridSearchCV(estimator=classificador['classifier'],
                                param_grid=classificador['parameters'],
                                scoring='roc_auc',
                                verbose=False,
                                cv=5,
                                n_jobs=10)

    gs.fit(data_vect, data['SMARTPHONE'])
    resultados[classificador['name']] = {'modelo': gs.best_estimator_, 'score': gs.best_score_}

In [43]:
resultados

{'svm': {'modelo': SVC(C=15, cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
      max_iter=-1, probability=False, random_state=None, shrinking=True,
      tol=0.001, verbose=False), 'score': 0.9997100172714589},
 'decision_tree': {'modelo': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                         max_features=None, max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=2,
                         min_weight_fraction_leaf=0.0, presort=False,
                         random_state=None, splitter='best'),
  'score': 0.9758512495929595},
 'bernoulli_naive_bayes': {'modelo': BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
  'score': 0.9961516980654448},
 'multinomial_naive_bayes': {'modelo': MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),

In [41]:
resultados['svm']['modelo']

SVC(C=15, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Parte 4: Classificação